In [ ]:
!wget -nc http://groups.csail.mit.edu/vision/LabelMe/NewImages/indoorCVPR_09.tar

--2020-08-19 07:50:27--  http://groups.csail.mit.edu/vision/LabelMe/NewImages/indoorCVPR_09.tar
Resolving groups.csail.mit.edu (groups.csail.mit.edu)... 128.30.2.44
Connecting to groups.csail.mit.edu (groups.csail.mit.edu)|128.30.2.44|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2592010240 (2.4G) [application/x-tar]
Saving to: ‘indoorCVPR_09.tar’

indoorCVPR_09.tar   100%[===================>]   2.41G  6.72MB/s    in 4m 11s  

2020-08-19 07:54:38 (9.86 MB/s) - ‘indoorCVPR_09.tar’ saved [2592010240/2592010240]



In [ ]:

!tar -xvf indoorCVPR_09.tar

Streaming output truncated to the last 5000 lines.
Images/mall/CP67791.jpg
Images/mall/crocker_galleria_735089.jpg
Images/mall/mall1.jpg
Images/mall/d8.jpg
Images/mall/deira_city_centre_dubai_08.jpg
Images/mall/dungarv_new_mall.jpg
Images/mall/latham_circle_mall_16.jpg
Images/mall/eastridg1.jpg
Images/mall/IMG_6449.jpg
Images/mall/main_boutique_b.jpg
Images/mall/easyfranchise_centre_commercial_1.jpg
Images/mall/ins42.jpg
Images/mall/mall14.jpg
Images/mall/eaton_centre_2001_03.jpg
Images/mall/ins43.jpg
Images/mall/mall17.jpg
Images/mall/full_shopping_Central_1.jpg
Images/mall/galleria.jpg
Images/mall/galleria1.jpg
Images/mall/galleria2.jpg
Images/mall/Galleria_003.jpg
Images/mall/laurel_mall_06.jpg
Images/mall/Galleria_1.jpg
Images/mall/galleria_san_federico_nr_109_2_n1.jpg
Images/mall/ins23.jpg
Images/mall/ins44.jpg
Images/mall/mall18.jpg
Images/mall/galleria_vittorio_emanuele_ii_milan_mlgalar.jpg
Images/mall/ins45.jpg
Images/mall/mall19.jpg
Images/mall/GUM_Department_Store.jpg
Images/

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time


In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
train_transforms = transforms.Compose([transforms.Resize(224),
                                       transforms.CenterCrop(224),
                                       transforms.RandomRotation(30),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomVerticalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])


In [ ]:
train_data = datasets.ImageFolder(root = '/content/Images',transform=train_transforms)

In [ ]:
VALID_RATIO = 0.35

n_train_examples = int(len(train_data) * VALID_RATIO)
n_valid_examples = len(train_data) - n_train_examples
print(n_train_examples)
print(n_valid_examples)

train_data, valid_data = data.random_split(train_data, 
                                           [n_train_examples, n_valid_examples])

5467
10153


In [ ]:
VALID_RATIO = 0.9

n_train_examples = int(len(train_data) * VALID_RATIO)
n_valid_examples = len(train_data) - n_train_examples
print(n_train_examples)
print(n_valid_examples)

train_data, valid_data = data.random_split(train_data, 
                                           [n_train_examples, n_valid_examples])

4920
547


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import copy
import random
import time
import torchvision.models as models

In [ ]:
model = models.resnet101(pretrained=True).to(device)

for param in model.parameters():
  param.requires_grad=True
    
model.fc = nn.Sequential(
               nn.Linear(2048, 496),
               nn.ReLU(inplace=True),
               nn.Linear(496, 67)).to(device)

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


In [ ]:
BATCH_SIZE = 16
train_iterator = data.DataLoader(train_data, 
                                 shuffle = True, 
                                 batch_size = BATCH_SIZE)

valid_iterator = data.DataLoader(valid_data, 
                                 batch_size = BATCH_SIZE)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 43,549,763 trainable parameters


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0, dampening=0, weight_decay=0, nesterov=False)

criterion = criterion.to(device)

In [ ]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for (x, y) in iterator:
        
        x = x.to(device)
        y = y.to(device)
        y = y.type(torch.LongTensor)
        y = y.to(device)
        
        optimizer.zero_grad()
                
        y_pred = model(x)
        
        loss = criterion(y_pred, y)
        
        acc = calculate_accuracy(y_pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        
        for (x, y) in iterator:

            x = x.to(device)
            y = y.to(device)
            y = y.type(torch.LongTensor)
            y = y.to(device)

            y_pred = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
EPOCHS = 200

best_valid_loss = float('inf')

for epoch in range(EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)
    
    if valid_loss < best_valid_loss :
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 4m 47s
	Train Loss: 3.942 | Train Acc: 8.89%
	 Val. Loss: 3.720 |  Val. Acc: 12.50%
Epoch: 02 | Epoch Time: 4m 48s
	Train Loss: 3.428 | Train Acc: 20.98%
	 Val. Loss: 3.046 |  Val. Acc: 25.24%
Epoch: 03 | Epoch Time: 4m 48s
	Train Loss: 2.870 | Train Acc: 30.58%
	 Val. Loss: 2.545 |  Val. Acc: 36.67%
Epoch: 04 | Epoch Time: 4m 48s
	Train Loss: 2.413 | Train Acc: 38.84%
	 Val. Loss: 2.146 |  Val. Acc: 46.49%
Epoch: 05 | Epoch Time: 4m 48s
	Train Loss: 2.076 | Train Acc: 46.29%
	 Val. Loss: 1.851 |  Val. Acc: 49.88%
Epoch: 06 | Epoch Time: 4m 47s
	Train Loss: 1.791 | Train Acc: 52.05%
	 Val. Loss: 1.675 |  Val. Acc: 55.24%
Epoch: 07 | Epoch Time: 4m 49s
	Train Loss: 1.595 | Train Acc: 56.33%
	 Val. Loss: 1.514 |  Val. Acc: 58.45%
Epoch: 08 | Epoch Time: 4m 49s
	Train Loss: 1.390 | Train Acc: 62.26%
	 Val. Loss: 1.456 |  Val. Acc: 59.35%
Epoch: 09 | Epoch Time: 4m 49s
	Train Loss: 1.247 | Train Acc: 65.34%
	 Val. Loss: 1.359 |  Val. Acc: 62.62%
Epoch: 10 | Epoch Ti

KeyboardInterrupt: ignored